# IIC：Invariant Information Clustering for Unsupervised Image Classification and Segmentation

参考：
https://github.com/RuABraun/phone-clustering


In [0]:
# 乱数のシードを固定
import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合

In [0]:
# GPUが使えるときにはGPUに（Google Colaboratoryの場合はランタイムからGPUを指定）
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)  

# GPUを使用。cudaと出力されるのを確認する。


cuda


In [0]:
# MNISTの画像をダウンロードし、DataLoaderにする（TrainとTest）
from torchvision import datasets, transforms

batch_size_train = 512

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                   ])),
    batch_size=batch_size_train, shuffle=True, drop_last=True)
# drop_lastは最後のミニバッチが規定のサイズより小さい場合は使用しない設定


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
    ])),
    batch_size=1024, shuffle=False)


In [0]:
# ディープラーニングモデル
import torch.nn as nn
import torch.nn.functional as F

OVER_CLUSTRING_Rate = 10  # 多めに分類するoverclsuteringも用意する


class NetIIC(nn.Module):
    def __init__(self):
        super(NetIIC, self).__init__()

        self.conv1 = nn.Conv2d(1, 128, 5, 2, bias=False)
        self.bn1 = nn.BatchNorm2d(128)
        self.conv2 = nn.Conv2d(128, 128, 5, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 128, 5, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 4, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(256)
        
        # 0-9に対応すると期待したい10種類のクラス
        self.fc = nn.Linear(256, 10)

        # overclustering
        # 実際の想定よりも多めにクラスタリングさせることで、ネットワークで微細な変化を捉えられるようにする
        self.fc_overclustering = nn.Linear(256, 10*OVER_CLUSTRING_Rate)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x_prefinal = x.view(x.size(0), -1)
        y = F.softmax(self.fc(x_prefinal), dim=1)

        y_overclustering = F.softmax(self.fc_overclustering(
            x_prefinal), dim=1)  # overclustering

        return y, y_overclustering


In [0]:
import torch.nn.init as init


def weight_init(m):
    """重み初期化"""
    if isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        # Xavier
        #init.xavier_normal_(m.weight.data)

        # He 
        init.kaiming_normal_(m.weight.data)
        
        if m.bias is not None:
            init.normal_(m.bias.data)


In [0]:
# データにノイズを加える関数の定義
import torchvision as tv
import torchvision.transforms.functional as TF


def perturb_imagedata(x):
    y = x.clone()
    batch_size = x.size(0)

    # ランダムなアフィン変換を実施
    trans = tv.transforms.RandomAffine(15, (0.2, 0.2,), (0.2, 0.75,))
    for i in range(batch_size):
        y[i, 0] = TF.to_tensor(trans(TF.to_pil_image(y[i, 0])))

    # ノイズを加える
    noise = torch.randn(batch_size, 1, x.size(2), x.size(3))
    div = torch.randint(20, 30, (batch_size,),
                        dtype=torch.float32).view(batch_size, 1, 1, 1)
    y += noise / div

    return y


In [0]:
# IISによる損失関数の定義
# 参考：https://github.com/RuABraun/phone-clustering/blob/master/mnist_basic.py
import sys


def compute_joint(x_out, x_tf_out):

    # x_out、x_tf_outは torch.Size([512, 10])。この二つをかけ算して同時分布を求める、torch.Size([2048, 10, 10])にする。
    # torch.Size([512, 10, 1]) * torch.Size([512, 1, 10])
    p_i_j = x_out.unsqueeze(2) * x_tf_out.unsqueeze(1)
    # p_i_j は　torch.Size([512, 10, 10])

    # 全ミニバッチを足し算する ⇒ torch.Size([10, 10])
    p_i_j = p_i_j.sum(dim=0)

    # 転置行列と足し算して割り算（対称化） ⇒ torch.Size([10, 10])
    p_i_j = (p_i_j + p_i_j.t()) / 2.

    # 規格化 ⇒ torch.Size([10, 10])
    p_i_j = p_i_j / p_i_j.sum()

    return p_i_j
    # 結局、p_i_jは通常画像の判定出力10種類と、変換画像の判定10種類の100パターンに対して、全ミニバッチが100パターンのどれだったのかの確率分布表を示す


def IID_loss(x_out, x_tf_out, EPS=sys.float_info.epsilon):
    # torch.Size([512, 10])、後ろの10は分類数なので、overclusteringのときは100
    bs, k = x_out.size()
    p_i_j = compute_joint(x_out, x_tf_out)  # torch.Size([10, 10])

    # 同時確率の分布表から、変換画像の10パターンをsumをして周辺化し、元画像だけの周辺確率の分布表を作る
    p_i = p_i_j.sum(dim=1).view(k, 1).expand(k, k)
    # 同時確率の分布表から、元画像の10パターンをsumをして周辺化し、変換画像だけの周辺確率の分布表を作る
    p_j = p_i_j.sum(dim=0).view(1, k).expand(k, k)

    # 0に近い値をlogに入れると発散するので、避ける
    #p_i_j[(p_i_j < EPS).data] = EPS
    #p_j[(p_j < EPS).data] = EPS
    #p_i[(p_i < EPS).data] = EPS
    # 参考GitHubの実装（↑）は、PyTorchのバージョン1.3以上だとエラーになる
    # https://discuss.pytorch.org/t/pytorch-1-3-showing-an-error-perhaps-for-loss-computed-from-paired-outputs/68790/3

    # 0に近い値をlogに入れると発散するので、避ける
    p_i_j = torch.where(p_i_j < EPS, torch.tensor(
        [EPS], device=p_i_j.device), p_i_j)
    p_j = torch.where(p_j < EPS, torch.tensor([EPS], device=p_j.device), p_j)
    p_i = torch.where(p_i < EPS, torch.tensor([EPS], device=p_i.device), p_i)

    # 元画像、変換画像の同時確率と周辺確率から、相互情報量を計算
    # ただし、マイナスをかけて最小化問題にする
    """
    相互情報量を最大化したい
    ⇒結局、x_out, x_tf_outが持ちあう情報量が多くなって欲しい
    ⇒要は、x_out, x_tf_outが一緒になって欲しい

    p_i_jはx_out, x_tf_outの同時確率分布で、ミニバッチが極力、10×10のいろんなパターン、満遍なく一様が嬉しい
    
    前半の項、torch.log(p_i_j)はp_ijがどれも1に近いと大きな値（0に近い）になる。
    どれかが1であと0でバラついていないと、log0で小さな値（負の大きな値）になる
    つまり前半の項は、

    後半の項は、元画像、もしくは変換画像について、それぞれ周辺化して10通りのどれになるかを計算した項。
    周辺化した10×10のパターンを引き算して、前半の項が小さくなるのであれば、
    x_outとx_tf_outはあまり情報を共有していなかったことになる。
    """
    # https://qiita.com/Amanokawa/items/0aa24bc396dd88fb7d2a
    # を参考に、重みalphaを追加
    # 同時確率分布表のばらつきによる罰則を小さく ＝ 同時確率の分布がバラつきやすくする
    alpha = 2.0  # 論文や通常の相互情報量の計算はalphaは1です

    loss = -1*(p_i_j * (torch.log(p_i_j) - alpha *
                        torch.log(p_j) - alpha*torch.log(p_i))).sum()

    return loss


In [0]:
# 訓練の実施
total_epoch = 20


# モデル
model = NetIIC()
model.apply(weight_init)
model.to(device)

# 最適化関数を設定
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


def train(total_epoch, model, train_loader, optimizer, device):

    model.train()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=2, T_mult=2)

    for epoch in range(total_epoch):
        for batch_idx, (data, target) in enumerate(train_loader):

            # 学習率変化
            scheduler.step()

            # 微妙に変換したデータを作る。SIMULTANEOUS_NUM分のペアを作る
            data_perturb = perturb_imagedata(data)  # ノイズを与える

            # GPUに送れる場合は送る
            data = data.to(device)
            data_perturb = data_perturb.to(device)

            # 最適化関数の初期化
            optimizer.zero_grad()

            # ニューラルネットワーク出力
            output, output_overclustering = model(data)
            output_perturb, output_perturb_overclustering = model(data_perturb)

            # 損失の計算
            loss1 = IID_loss(output, output_perturb)
            loss2 = IID_loss(output_overclustering,
                             output_perturb_overclustering)
            loss = loss1 + loss2

            # 損失を減らすように更新
            loss.backward()
            optimizer.step()

            # ログ出力
            if batch_idx % 10 == 0:
                print('Train Epoch {}:iter{} - \tLoss1: {:.6f}- \tLoss2: {:.6f}- \tLoss_total: {:.6f}'.format(
                    epoch, batch_idx, loss1.item(), loss2.item(), loss1.item()+loss2.item()))

    return model, optimizer


model_trained, optimizer = train(
    total_epoch, model, train_loader, optimizer, device)


Train Epoch 0:iter0 - 	Loss1: -4.061602- 	Loss2: -7.873592- 	Loss_total: -11.935194
Train Epoch 0:iter10 - 	Loss1: -4.624140- 	Loss2: -9.001599- 	Loss_total: -13.625739
Train Epoch 0:iter20 - 	Loss1: -4.639630- 	Loss2: -9.182800- 	Loss_total: -13.822430
Train Epoch 0:iter30 - 	Loss1: -4.642681- 	Loss2: -9.222592- 	Loss_total: -13.865273
Train Epoch 0:iter40 - 	Loss1: -4.686302- 	Loss2: -9.257124- 	Loss_total: -13.943426
Train Epoch 0:iter50 - 	Loss1: -4.694422- 	Loss2: -9.263514- 	Loss_total: -13.957935
Train Epoch 0:iter60 - 	Loss1: -4.716135- 	Loss2: -9.279631- 	Loss_total: -13.995765
Train Epoch 0:iter70 - 	Loss1: -4.792006- 	Loss2: -9.310069- 	Loss_total: -14.102076
Train Epoch 0:iter80 - 	Loss1: -4.855150- 	Loss2: -9.382927- 	Loss_total: -14.238077
Train Epoch 0:iter90 - 	Loss1: -5.007874- 	Loss2: -9.461571- 	Loss_total: -14.469445
Train Epoch 0:iter100 - 	Loss1: -5.016116- 	Loss2: -9.506951- 	Loss_total: -14.523067
Train Epoch 0:iter110 - 	Loss1: -5.076643- 	Loss2: -9.549101- 	Lo

In [0]:
# モデル分類のクラスターの結果を確認する

def test(model, device, train_loader):
    model.eval()

    # 結果を格納するリスト
    out_targs = []
    ref_targs = []
    cnt = 0

    with torch.no_grad():
        for data, target in test_loader:
            cnt += 1
            data = data.to(device)
            target = target.to(device)
            outputs, outputs_overclustering = model(data)

            # 分類結果をリストに追加
            out_targs.append(outputs.argmax(dim=1).cpu())
            ref_targs.append(target.cpu())

    # リストをひとまとめに
    out_targs = torch.cat(out_targs)
    ref_targs = torch.cat(ref_targs)

    return out_targs.numpy(), ref_targs.numpy()


out_targs, ref_targs = test(model_trained, device, train_loader)


In [0]:
import numpy as np
import scipy.stats as stats

# 混同行列（的な）を作る
matrix = np.zeros((10, 10))

# 縦に数字の0から9を、横に判定されたクラスの頻度表を作成
for i in range(len(out_targs)):
    row = ref_targs[i]
    col = out_targs[i]
    matrix[row][col] += 1

np.set_printoptions(suppress=True)
print(matrix)


[[   1.  978.    1.    0.    0.    0.    0.    0.    0.    0.]
 [   1.    0.    4. 1110.    2.    0.    2.    0.   13.    3.]
 [   0.    5.    4.    0.    0.    0.    0.    0. 1023.    0.]
 [   0.    0.    2.    0.    0.    4.  962.    0.   39.    3.]
 [   1.    0.    0.    0.  960.    0.    0.   19.    1.    1.]
 [   1.    1.    0.    0.    0.  866.   17.    0.    3.    4.]
 [ 940.    7.    0.    0.    0.    3.    0.    0.    4.    4.]
 [   0.    0.  921.    1.    0.    0.    1.   92.   13.    0.]
 [   0.    6.    0.    0.    0.    2.    4.    2.    2.  958.]
 [   0.    4.   14.    0.    2.    7.   27.  949.    2.    4.]]


In [0]:
# 全データ
total_num = matrix.sum().sum()
print(total_num)

# 各数字がきれいに各クラスに分かれている。
# 例えば数字の0はクラスの1番目に978個集まった。数字の9であれば、7番目に949個集まった。
# よって、最大のものを足していくと、正解の個数なので
correct_num_list = matrix.max(axis=0)
print(correct_num_list)
print(correct_num_list.sum())

print("正解率：", correct_num_list.sum()/total_num*100)


10000.0
[ 940.  978.  921. 1110.  960.  866.  962.  949. 1023.  958.]
9667.0
正解率： 96.67


以上。